In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from os import listdir
import numpy as np
import math
from scipy.spatial import ConvexHull
from scipy.spatial.distance import cdist
from scipy.spatial import distance
from itertools import groupby

In [19]:
def calculateVolume(data):
#      multiply by constant 997
    result = 0
    for d in data:
        result += (1/6)*math.pi*math.pow(d,3)* 917
#     return round(result, 3)
    return result

In [20]:
def calculateLength(data):
    """
    find the most distant point
    data = a nupmpy array containing the points
    """
    """first check if the x points are same or not
    if same we will use the naive techniques
    if not we will use the convex hull approach
    """
    xVals = [i[0] for i in data]
#     print(xVals)
    g = groupby(xVals)
    sameValues = next(g, True) and not next(g, False)
#     print(sameValues)
    
    if sameValues == False:
        # # Find a convex hull in O(N log N)
        hull = ConvexHull(data)

        # Extract the points forming the hull
        hullpoints = data[hull.vertices,:]
    
    elif sameValues == True:
        hullpoints = data       

#     print(hullpoints)
    
    # Naive way of finding the best pair in O(H^2) time if H is number of points on
    # hull
    hdist = cdist(hullpoints, hullpoints, metric='euclidean')

    # Get the farthest apart points
    bestpair = np.unravel_index(hdist.argmax(), hdist.shape)

    #Print them
    points = [hullpoints[bestpair[0]],hullpoints[bestpair[1]]]
#     print(points)

    # print the distance
#     dist = np.linalg.norm(points[0] - points[1])
#     print(dist)
    dst = distance.euclidean(points[0], points[1])
    return round(dst, 3)

In [21]:
columns = ["cluster", "temperature", "particles", "volumes", "distance"]
folders = [f for f in listdir("data/")]
for folder in folders:
    onlyfiles = [f for f in listdir("data/{}".format(folder)) ]
#     print(folder)
    for file in onlyfiles:
        path = "data/{}/{}".format(folder, file)
        df = pd.read_csv(path)
#             calculating the diameter in micron (?)
        df['new_d'] = df['d'] * math.pow(10, 6)
    
#         getting the temperature means
        means = round(df.groupby('cluster')['T'].mean(), 3)
#     getting the clusters
        clusters = means.index.tolist()
#     storing the temperature
        temperature = means.tolist()
#     storing total particles
        particles = df.groupby('cluster')['cluster'].value_counts().tolist()

#     adding the total values for all data
        clusters.append("all")
        temperature.append(round(df['T'].mean(), 3))
        particles.append(df.shape[0])

#         extracting the total volumes
        volumes = []
        volumesExact = []
#         extracting the farthest distance
        distances = []
        for each in clusters:
#             print(each)
            if each == 'all':
                volumes.append(calculateVolume(df['new_d'].tolist()))
                volumesExact.append(calculateVolume(df['d'].tolist()))
                distances.append(calculateLength(df[['Points:0', 'Points:1', 'Points:2']].to_numpy()))
            else:
                volumes.append(calculateVolume(df[df.cluster == each]['new_d'].tolist()))
                volumesExact.append(calculateVolume(df[df.cluster == each]['d'].tolist()))
                distances.append(calculateLength(df[df.cluster == each][['Points:0', 'Points:1', 'Points:2']].to_numpy()))
        
        
        # print(means)
        print(clusters)
#         print(temperature)
#         print(particles)
#         print(volumes)
#         print(distances)
        print(volumesExact)

# output csv generation
        extract = pd.DataFrame(np.column_stack([clusters, temperature, particles, volumes, distances]),
                      columns=columns)

        extract.to_csv('output/{}/{}'.format(folder, file), index=False)
        print("exported {} {} ".format(folder, file))


[1, 2, 3, 4, 5, 'all']
[1.3469086708670726e-09, 1.5606126483606208e-10, 6.16057629163638e-11, 9.598590577690529e-11, 2.5341647172141318e-08, 2.7002208776537657e-08]
exported contrails1 0.1.csv 
[1, 2, 3, 4, 5, 'all']
[2.8818638212497133e-08, 8.33493764647051e-11, 1.0202547949228774e-10, 1.879259140137395e-11, 3.772382125003901e-11, 2.906052948110528e-08]
exported contrails1 0.11.csv 
[1, 'all']
[2.9893554646749274e-08, 2.9893554646749274e-08]
exported contrails1 0.12.csv 
[1, 2, 'all']
[3.008695613972775e-08, 3.7285592064975854e-11, 3.01242417317927e-08]
exported contrails1 0.13.csv 
[1, 2, 3, 4, 'all']
[9.387286481258388e-10, 1.4042109807133584e-10, 5.7744550470002016e-11, 2.8721674254498986e-08, 2.9858568551166705e-08]
exported contrails1 0.14.csv 
[1, 2, 3, 'all']
[2.9590752856915777e-08, 1.1587207878847244e-10, 1.8360510023578023e-11, 2.9724985445727972e-08]
exported contrails1 0.15.csv 
[1, 2, 3, 4, 'all']
[2.9104040238042215e-08, 5.846288616538452e-11, 1.8692285205032492e-11, 1.8

In [10]:
path = "data/withoutOutliers/{}/{}".format("contrails1", "0.2.csv")
df = pd.read_csv(path)
pts = df[['Points:0', 'Points:1', 'Points:2']].to_numpy()
hull = ConvexHull(p)

In [14]:

from mpl_toolkits.mplot3d import Axes3D


# 8 points defining the cube corners
# pts = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0],
#                 [0, 0, 1], [1, 0, 1], [1, 1, 1], [0, 1, 1], ])

hull = ConvexHull(pts)

# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")

# # Plot defining corner points
# ax.plot(pts.T[0], pts.T[1], pts.T[2], "ko")

# # 12 = 2 * 6 faces are the simplices (2 simplices per square face)
# for s in hull.simplices:
#     s = np.append(s, s[0])  # Here we cycle back to the first coordinate
#     ax.plot(pts[s, 0], pts[s, 1], pts[s, 2], "r-")

# # Make axis label
# for i in ["x", "y", "z"]:
#     eval("ax.set_{:s}label('{:s}')".format(i, i))

# plt.show()

print(hull)